<a href="https://colab.research.google.com/github/aryatomarAI/Machine-learning-and-data-science/blob/main/Deep-Learning-plus-Project/Dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐕‍🦺 Using Tensorflow 2.0 and Transfer learning to classify different dog breeds
This notebook builds an end-to-end multi-class image classifier using Tensorflow and Tensorflow Hub

We're using Kaggle Dog Identification competition
## 1. Problem
   Identifying the breed of a dog given an image of a dog.
## 2. Data
   The data we're using is from [Kaggle's dog breed identification competition](https://www.kaggle.com/c/dog-breed-identification/data)
## 3. Evaluation
   The evaluation is a file with prediction probabilities for each dog breed of each test image.

   [Evaluated on Multi Class Log Loss between the predicted probability and the observed target](https://www.kaggle.com/c/dog-breed-identification/overview/evaluation)

## 4. Features

Some Information about the data:

We're dealing with images (Unstructured data).
It consists of a collection of 10,000+ labelled images of 120 different dog breeds.

This kind of problem is called multi-class image classification. It's multi-class because we're trying to classify mutliple different breeds of dog. If we were only trying to classify dogs versus cats, it would be called binary classification (one thing versus another).

Since the most important step in a deep learng problem is getting the data ready (turning it into numbers), that's what we're going to start with.

We're going to go through the following TensorFlow/Deep Learning workflow:
1. Get data ready (download from Kaggle, store, import).
2. Prepare the data (preprocessing, the 3 sets, X & y).
3. Choose and fit/train a model ([TensorFlow Hub](https://www.tensorflow.org/hub), `tf.keras.applications`, [TensorBoard](https://www.tensorflow.org/tensorboard), [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)).
4. Evaluating a model (making predictions, comparing them with the ground truth labels).
5. Improve the model through experimentation (start with 1000 images, make sure it works, increase the number of images).
6. Save, sharing and reloading your model (once you're happy with the results).

For preprocessing our data, we're going to use TensorFlow 2.x. The whole premise here is to get our data into Tensors (arrays of numbers which can be run on GPUs) and then allow a machine learning model to find patterns between them.

For our machine learning model, we're going to be using a pretrained deep learning model from TensorFlow Hub.




## Get our Workspace ready

In [2]:
# Import Tensorflow into colab
import tensorflow as tf
print("We are using Tensor flow version",tf.__version__)

We are using Tensor flow version 2.4.1


In [3]:
# Importing Tensflow_hub and checking for GPU 
import tensorflow_hub as hub
print("Tensorflow hub version:",hub.__version__)

print("GPU","available" if tf.config.list_physical_devices("GPU") else "not available")

Tensorflow hub version: 0.12.0
GPU available
